In [1]:
!pip install torch torchvision numpy


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import torchvision
import numpy as np
import torch
from torch.utils.data import DataLoader
import json
import os

### Download pre-trained model

In [3]:
pretrained_model = torchvision.models.resnet34(weights='IMAGENET1K_V1')

### Setup dataset

In [4]:
!unzip -n TestDataSet.zip 

Archive:  TestDataSet.zip


In [23]:
mean_norms = np.array([0.485, 0.456, 0.406])
std_norms = np.array([0.229, 0.224, 0.225])
plain_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(), 
    torchvision.transforms.Normalize(mean=mean_norms, std=std_norms) 
])

batch_size = 128
dataset_path = "./TestDataSet"
dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=plain_transforms)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [27]:
# Load label mapping
with open(dataset_path + '/labels_list.json', "r") as f:
    label_list = json.load(f)

class_idx = [int(label.split(': ')[0]) for label in label_list]
class_label = [label.split(': ')[1] for label in label_list]

# map index to labels
class_mapping = {idx: label for idx, label in zip(class_idx, class_label)}

# map folders to labels
subfolders = [ f.name for f in os.scandir(dataset_path) if f.is_dir() ]
subfolders.sort()
subfolders

folder_map = {folder_name: idx for folder_name, idx in zip(subfolders, class_idx)}

target_to_imagenet = {target: idx for target, idx in enumerate(class_idx)}

### Evaluate pre-trained model

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pretrained_model = pretrained_model.to(device)
pretrained_model.eval()

top1_correct = 0
top5_correct = 0
total = 0

with torch.no_grad():
    for images, targets in dataloader:
        images = images.to(device)
        targets = targets.to(device)
        # map targets from test dataset to imagenet predictions
        imagenet_targets = torch.tensor(
            [target_to_imagenet[int(t)] for t in targets], device=device
        )

        # get preds
        outputs = pretrained_model(images)
        _, top5_preds = outputs.topk(5, dim=1)

        # Top-1 accuracy
        top1_correct += (top5_preds[:, 0] == imagenet_targets).sum().item()

        # Top-5 accuracy
        for i in range(images.size(0)):
            if imagenet_targets[i].item() in top5_preds[i]:
                top5_correct += 1

        total += images.size(0)

# Compute accuracy
top1_acc = top1_correct / total
top5_acc = top5_correct / total

print(f"Top-1 Accuracy: {top1_acc * 100:.2f}%")
print(f"Top-5 Accuracy: {top5_acc * 100:.2f}%")

Top-1 Accuracy: 76.00%
Top-5 Accuracy: 94.20%
